In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import getpass
import os
from pymongo import MongoClient
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.document_loaders import DirectoryLoader
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import gradio as gr
from gradio.themes.base import Base
import key_param

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] =  key_param.GEMINI_API_KEY #getpass.getpass("Enter your Google AI API key: ")

In [2]:
client = MongoClient(key_param.MONGO_URI)
dbName = "langchain_demo"
collectionName = "collection_of_text_blobs"
collection = client[dbName][collectionName]

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", task_type="retrieval_query")

vectorStore = MongoDBAtlasVectorSearch(embedding = embeddings,index_name = "sahil", collection=collection )

C:\Users\sahil\AppData\Local\Temp\ipykernel_12448\3246634906.py:8: LangChainDeprecationWarning: The class `MongoDBAtlasVectorSearch` was deprecated in LangChain 0.0.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-mongodb package and should be used instead. To use it run `pip install -U :class:`~langchain-mongodb` and import as `from :class:`~langchain_mongodb import MongoDBAtlasVectorSearch``.
  vectorStore = MongoDBAtlasVectorSearch(embedding = embeddings,index_name = "sahil", collection=collection )


In [3]:
def query_data(query): 

    docs = vectorStore.similarity_search(query, K=1)
    as_output = docs[0].page_content

    llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=20,
    timeout=None,
    max_retries=2,
)
    retriever = vectorStore.as_retriever()

    qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)


    retriever_output = qa.run(query)
    return as_output, retriever_output

In [4]:
with gr.Blocks(theme=Base(), title="Question Answering App using Vector Search + RAG") as demo:
    gr.Markdown(
        """
        # Talk to your Documents
        """)
    textbox = gr.Textbox(label="Enter your Question:")
    with gr.Row():
        button = gr.Button("Submit", variant="primary")
    with gr.Column():
        output1 = gr.Textbox(lines=1, max_lines=10, label="Output with just Atlas Vector Search (returns text field as is):")
        output2 = gr.Textbox(lines=1, max_lines=10, label="Output generated by chaining Atlas Vector Search to Langchain's RetrieverQA + GoogleAI LLM:")


    button.click(query_data, textbox, outputs=[output1, output2])

demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
